In [2]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'test-00000-of-00001.parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [3]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,question,answer
0,consensus,Solve the following math word problem.\n\nJane...,Solve the following math word problem.\n\nJane...,[18],[18],math,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...
1,consensus,Solve the following math word problem.\n\nA ro...,Solve the following math word problem.\n\nA ro...,[3],[3],math,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...
2,verified,Solve the following math word problem.\n\nJosh...,Solve the following math word problem.\n\nJosh...,[70000],[70000],math,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...
3,consensus,Solve the following math word problem.\n\nJame...,Solve the following math word problem.\n\nJame...,[540],[540],math,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...
4,rejected,Solve the following math word problem.\n\nEver...,Solve the following math word problem.\n\nEver...,None,[20],math,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t..."


In [4]:
df['platinum_prompt_no_cot'].value_counts()

platinum_prompt_no_cot
Solve the following math word problem.\n\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n\nThen, provide the final answer as a single integer in the format "Answer: XXX" with no extra formatting.                                                                                                                              1
Solve the following math word problem.\n\nMicheal loves riding a bike. He rode it at least 5 times a week and makes 25 kilometers each time. He did that for four weeks, and then he decided, to ride the bike only 2 times a week, but for 60 kilometers each time, and he did that for 3 weeks. How many kilometers did Micheal do in total?\n\nThen, provide the final answer as a single integer in the format "Answer: XXX" wi

In [6]:
import re

def double_string(match):
    return match.group(1) + "Read the question again: " + match.group(1).lstrip()

df['RE2'] = df['platinum_prompt'].apply(lambda x: re.sub(r'(\n\n.*?\n\n)', double_string, x))
df['RE2'].value_counts()

RE2
Solve the following math word problem.\n\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n\nRead the question again: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n\nThink step-by-step. Then, provide the final answer as a single integer in the format "Answer: XXX" with no extra formatting.                                                                                                                                                                                                                                             

In [7]:
def add_read_task(match):
    return match.group(1) + "Read the question again first. "

In [8]:
df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].apply(lambda x: re.sub(r'(\n\n.*?\n\n)', add_read_task, x))
df['sRE2'] = df['platinum_prompt'].apply(lambda x: re.sub(r'(\n\n.*?\n\n)', add_read_task, x))

In [9]:
df['sRE2'].value_counts()

sRE2
Solve the following math word problem.\n\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n\nRead the question again first. Think step-by-step. Then, provide the final answer as a single integer in the format "Answer: XXX" with no extra formatting.                                                                                                                              1
Solve the following math word problem.\n\nMicheal loves riding a bike. He rode it at least 5 times a week and makes 25 kilometers each time. He did that for four weeks, and then he decided, to ride the bike only 2 times a week, but for 60 kilometers each time, and he did that for 3 weeks. How many kilometers did Micheal do in total?\n\nRead the question again first. Think step-by-ste

In [10]:
def add_sum_up_task(match):
    return match.group(1) + "Summarize the question first. "

In [11]:
df['sum_no_cot'] = df['platinum_prompt_no_cot'].apply(lambda x: re.sub(r'(\n\n.*?\n\n)', add_sum_up_task, x))
df['sum'] = df['platinum_prompt'].apply(lambda x: re.sub(r'(\n\n.*?\n\n)', add_sum_up_task, x))

In [12]:
df['sum'].value_counts()

sum
Solve the following math word problem.\n\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n\nSummarize the question first. Think step-by-step. Then, provide the final answer as a single integer in the format "Answer: XXX" with no extra formatting.                                                                                                                              1
Solve the following math word problem.\n\nMicheal loves riding a bike. He rode it at least 5 times a week and makes 25 kilometers each time. He did that for four weeks, and then he decided, to ride the bike only 2 times a week, but for 60 kilometers each time, and he did that for 3 weeks. How many kilometers did Micheal do in total?\n\nSummarize the question first. Think step-by-step. 

In [13]:
def add_table_task(match):
    return match.group(1) + "Organize the question into a table first. "

df['table_no_cot'] = df['platinum_prompt_no_cot'].apply(lambda x: re.sub(r'(\n\n.*?\n\n)', add_table_task, x))
df['table'] = df['platinum_prompt'].apply(lambda x: re.sub(r'(\n\n.*?\n\n)', add_table_task, x))

In [14]:
df['table'].value_counts()

table
Solve the following math word problem.\n\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n\nOrganize the question into a table first. Think step-by-step. Then, provide the final answer as a single integer in the format "Answer: XXX" with no extra formatting.                                                                                                                              1
Solve the following math word problem.\n\nMicheal loves riding a bike. He rode it at least 5 times a week and makes 25 kilometers each time. He did that for four weeks, and then he decided, to ride the bike only 2 times a week, but for 60 kilometers each time, and he did that for 3 weeks. How many kilometers did Micheal do in total?\n\nOrganize the question into a table f

In [15]:
def add_graph_task(match):
    return match.group(1) + "Organize the question into a graph structure first. "

df['graph_no_cot'] = df['platinum_prompt_no_cot'].apply(lambda x: re.sub(r'(\n\n.*?\n\n)', add_graph_task, x))
df['graph'] = df['platinum_prompt'].apply(lambda x: re.sub(r'(\n\n.*?\n\n)', add_graph_task, x))

In [16]:
df['graph'].value_counts()

graph
Solve the following math word problem.\n\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n\nOrganize the question into a graph structure first. Think step-by-step. Then, provide the final answer as a single integer in the format "Answer: XXX" with no extra formatting.                                                                                                                              1
Solve the following math word problem.\n\nMicheal loves riding a bike. He rode it at least 5 times a week and makes 25 kilometers each time. He did that for four weeks, and then he decided, to ride the bike only 2 times a week, but for 60 kilometers each time, and he did that for 3 weeks. How many kilometers did Micheal do in total?\n\nOrganize the question into

In [17]:
def add_bullet_point_task(match):
    return match.group(1) + "Summarize the question in bullet points first. "

df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].apply(lambda x: re.sub(r'(\n\n.*?\n\n)', add_bullet_point_task, x))
df['bullet_point'] = df['platinum_prompt'].apply(lambda x: re.sub(r'(\n\n.*?\n\n)', add_bullet_point_task, x))

In [18]:
df['bullet_point'].value_counts()

bullet_point
Solve the following math word problem.\n\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n\nSummarize the question in bullet points first. Think step-by-step. Then, provide the final answer as a single integer in the format "Answer: XXX" with no extra formatting.                                                                                                                              1
Solve the following math word problem.\n\nMicheal loves riding a bike. He rode it at least 5 times a week and makes 25 kilometers each time. He did that for four weeks, and then he decided, to ride the bike only 2 times a week, but for 60 kilometers each time, and he did that for 3 weeks. How many kilometers did Micheal do in total?\n\nSummarize the question i

In [19]:
df.to_parquet('gsm8k_add_task.parquet')